Language Model: a model of the probability of a sequence of words <br>
Ex. "The quick brown fox jumps over the lazy dog." <br>
A language model allows me to calculate: $P("\text{The quick brown fox jumps over the lazy dog.}")$

Bigram model: $P(w_t | w_{t-1})$. <br>
This is simple counting: $P(brown | quick) = \frac{count(quick \rightarrow brown)}{count(quick)}$

Need a set of documents for sentences to train the model.

Bayes Rule: $P("A B C") = P(C|A B)\times P(B|A)\times P(A)$, or called chain rule of probability.

Problem: if a sentence or word does not exist in the corpus but make sense, the model would still produce 0 probability.

**Add-one Smoothing**: instead of maximum-likelihood counting, add a small number to each count. <br>
Note: V := vocabulary size = number of distinct words <br>
$$P_{smooth}(B|A) = \frac{count(A \rightarrow B) + 1}{count(A) + V}$$
This still assigns a small probability even though it doesn't appear in corpus.

**The Markov Assumption**: what I see now depends *only* on what I saw in the previous step.
$$P(w_t|W_{t-1},\cdots,w_1) = P(w_t|w_{t-1})$$

First Markov Order Assumption: $P(E|A,B,C,D) = P(E|D)\times P(D|C)\times P(C|B)\times P(B|A)\times P(A)$

In [1]:
# import
from __future__ import print_function, division
from future.utils import iteritems
from builtins import range, input
import operator

import numpy as np

import os

from nltk.corpus import brown

In [2]:
KEEP_WORDS = set([
  'king', 'man', 'queen', 'woman',
  'italy', 'rome', 'france', 'paris',
  'london', 'britain', 'england',
])


def get_sentences():
    # returns 57340 of the Brown corpus
    # each sentence is represented as a list of individual string tokens
    return brown.sents()


def get_sentences_with_word2idx():
    sentences = get_sentences()
    indexed_sentences = []

    i = 2
    word2idx = {'START': 0, 'END': 1}
    for sentence in sentences:
        indexed_sentence = []
        for token in sentence:
            token = token.lower()
            if token not in word2idx:
                word2idx[token] = i
                i += 1
            indexed_sentence.append(word2idx[token])
        indexed_sentences.append(indexed_sentence)

    print("Vocab size:", i)
    return indexed_sentences, word2idx


def get_sentences_with_word2idx_limit_vocab(n_vocab=2000, keep_words=KEEP_WORDS):
    sentences = get_sentences()
    indexed_sentences = []

    i = 2
    word2idx = {'START': 0, 'END': 1}
    idx2word = ['START', 'END']

    word_idx_count = {
        0: float('inf'),
        1: float('inf'),
    }

    for sentence in sentences:
        indexed_sentence = []
        for token in sentence:
            token = token.lower()
            if token not in word2idx:
                idx2word.append(token)
                word2idx[token] = i
                i += 1

            # keep track of counts for later sorting
            idx = word2idx[token]
            word_idx_count[idx] = word_idx_count.get(idx, 0) + 1

            indexed_sentence.append(idx)
        indexed_sentences.append(indexed_sentence)

    # restrict vocab size

    # set all the words I want to keep to infinity
    # so that they are included when I pick the most
    # common words
    for word in keep_words:
        word_idx_count[word2idx[word]] = float('inf')

    sorted_word_idx_count = sorted(word_idx_count.items(), key=operator.itemgetter(1), reverse=True)
    word2idx_small = {}
    new_idx = 0
    idx_new_idx_map = {}
    for idx, count in sorted_word_idx_count[:n_vocab]:
        word = idx2word[idx]
        print(word, count)
        word2idx_small[word] = new_idx
        idx_new_idx_map[idx] = new_idx
        new_idx += 1
    # let 'unknown' be the last token
    word2idx_small['UNKNOWN'] = new_idx 
    unknown = new_idx

    assert('START' in word2idx_small)
    assert('END' in word2idx_small)
    for word in keep_words:
        assert(word in word2idx_small)

    # map old idx to new idx
    sentences_small = []
    for sentence in indexed_sentences:
        if len(sentence) > 1:
            new_sentence = [idx_new_idx_map[idx] if idx in idx_new_idx_map else unknown for idx in sentence]
            sentences_small.append(new_sentence)

    return sentences_small, word2idx_small

In [3]:
indexed_sentences, word2idx = get_sentences_with_word2idx()

Vocab size: 49817


In [4]:
V = len(word2idx)

In [5]:
start_idx = word2idx["START"]
end_idx = word2idx["END"]

In [6]:
# function for bigram probs
def get_bigram_probs(sentences, V, start_idx, end_idx, smoothing=1):
    # structure ofbigram prob matrix:
    #   (last_word_idx, current_word_idx) -> probabiliry
    # we will use add-1 smoothing
    
    bigram_probs = np.ones((V,V)) * smoothing
    for sentence in sentences:
        for i in range(len(sentence)):
            if i == 0:
                bigram_probs[start_idx, sentence[i]] += 1
            else:
                bigram_probs[sentence[i-1], sentence[i]] += 1
            
            if i == len(sentence) -1:
                bigram_probs[sentence[i], end_idx] += 1
        
    # normalize it
    bigram_probs /= bigram_probs.sum(axis=1, keepdims=True)
    return bigram_probs

In [7]:
bigram_probs = get_bigram_probs(indexed_sentences, V, 0, 1)

In [26]:
# score function
def get_score(sentence):
    score = 0
    for i in range(len(sentence)):
        if i == 0:
            score += np.log(bigram_probs[[start_idx], sentence[i]])
        else:
            score += np.log(bigram_probs[sentence[i-1], sentence[i]])
        
    score += np.log(bigram_probs[sentence[-1], end_idx])
    return score / (len(sentence) + 1)

In [43]:
idx2word = dict((v,k) for k, v in iteritems(word2idx))

# function to convert index back to real sentence
def get_words(sentence):
    return " ".join(idx2word[x] for x in sentence)

In [15]:
# sampling
start_idx = word2idx["START"]
end_idx = word2idx["END"]

sample_probs = np.ones(V)
sample_probs[start_idx] = 0
sample_probs[end_idx] = 0
sample_probs /= sample_probs.sum()

In [44]:
# a real sentence
real_idx = np.random.choice(len(indexed_sentences))
real = indexed_sentences[real_idx]
print(real)
print(get_words(real))
print(get_score(real))

[138, 1457, 6652, 23447, 41, 25798, 137, 25831, 44, 2, 3269, 22766, 1455, 1455]
is your sporting firearms and ammunition department primed for the expanding horizons ? ?
[-8.24288217]


In [45]:
# a fake sentence
fake = np.random.choice(V, size=len(real), p=sample_probs)
print(fake)
print(get_words(fake))
print(get_score(fake))

[12965 37763  7607 23441 32957 12083 42921 28736 48658 11557 10964 48990
 46639  3819]
nationalized belt-driven racket $350 seurat understandably blueberry nonobservant cutest reacting greet fire-colored beaded $457,000
[-10.8672143]
